### Importing Libraries

In [1]:
import pandas as pd
import numpy as np
pd.set_option("display.max_columns",None)

### Importing datasets

In [2]:
df_hltb = pd.read_parquet("jrpgs_db.parquet")
df_igdb = pd.read_parquet("games_data.parquet")

### Adding prefix in order to differentiate where the data came from

In [8]:
df_hltb = df_hltb.add_prefix('hltb_')
df_igdb = df_igdb.add_prefix('igdb_')

### Merging datasets

In [10]:
df = df_hltb.merge(df_igdb, how="left", left_on=["hltb_game_name", "hltb_release_world"], right_on=["igdb_nome", "igdb_ano"])

In [30]:
df.sample(5)

,hltb_game_id,hltb_game_name,hltb_game_name_date,hltb_game_alias,hltb_game_type,hltb_game_image,hltb_comp_lvl_combine,hltb_comp_lvl_sp,hltb_comp_lvl_co,hltb_comp_lvl_mp,hltb_comp_main,hltb_comp_plus,hltb_comp_100,hltb_comp_all,hltb_comp_main_count,hltb_comp_plus_count,hltb_comp_100_count,hltb_comp_all_count,hltb_invested_co,hltb_invested_mp,hltb_invested_co_count,hltb_invested_mp_count,hltb_count_comp,hltb_count_speedrun,hltb_count_backlog,hltb_count_review,hltb_review_score,hltb_count_playing,hltb_count_retired,hltb_profile_platform,hltb_profile_popular,hltb_release_world,igdb_nome,igdb_ano,igdb_name,igdb_slug,igdb_rating,igdb_rating_count,igdb_summary,igdb_url,igdb_franchises,igdb_game_modes,igdb_genres,igdb_keywords,igdb_platforms,igdb_player_perspectives,igdb_themes,igdb_collections,igdb_release_year
1399,3035,Mother,1,EarthBound Beginnings. EarthBound Zero,game,3035_Mother.png,0,1,0,0,60476,72270,84990,66340,139,66,28,233,0,0,0,0,513,0,1040,244,71,13,64,NES,84,1989,Mother,1989.0,Mother,mother--4,NaN,NaN,The original Japanese version of EarthBound Be...,https://www.igdb.com/games/mother--4,earthbound,single-player,role-playing-rpg,"virtual-console, japan-only, nintendo-switch-o...","wiiu, famicom",bird-view-slash-isometric,"fantasy, science-fiction",earthbound,NaN
1164,80403,Little Witch Nobeta,0,,game,80403_Little_Witch_Nobeta.jpg,0,1,0,0,23142,37042,50220,31416,26,29,9,64,0,0,0,0,127,0,296,46,74,7,22,"Nintendo Switch, PC, PlayStation 4",36,2020,Little Witch Nobeta,2020.0,Little Witch Nobeta,little-witch-nobeta,70.500000,2.0,"Control the lovely little witch Nobeta, explor...",https://www.igdb.com/games/little-witch-nobeta,None,single-player,"shooter, indie","anime, magic, difficult, female-protagonist, cute","ps4--1, win, switch",third-person,"action, comedy",None,NaN
369,38061,Death Stranding,0,,game,38061_Death_Stranding.jpg,0,1,0,0,145806,216097,406468,208349,1377,1875,624,3876,228865,0,1,0,6440,0,6616,2169,85,129,645,"PC, PlayStation 4",488,2019,Death Stranding,2019.0,Death Stranding,death-stranding,85.607294,687.0,From legendary game creator Hideo Kojima comes...,https://www.igdb.com/games/death-stranding,None,single-player,"shooter, role-playing-rpg, adventure","post-apocalyptic, kojima, e3-2016, the-game-aw...","ps4--1, win, ps5",third-person,"action, science-fiction, stealth, open-world",death-stranding,NaN
1122,5190,Last Rebellion,0,,game,Last_Rebellion_Boxart.jpg,0,1,1,1,40100,46061,47387,46299,4,3,12,19,0,0,0,0,32,0,81,13,47,0,4,PlayStation 3,5,2010,Last Rebellion,2010.0,Last Rebellion,last-rebellion,48.500000,5.0,"In Last Rebellion, assume the roles of Nine an...",https://www.igdb.com/games/last-rebellion,None,single-player,"shooter, role-playing-rpg","anime, turn-based, platform-exclusive, real-ti...",ps3,"third-person, side-view","action, science-fiction",None,NaN
1657,108270,Prinny Presents: NIS Classics - Volume 2,0,Prinny Presents NIS Classics Vol. 2,compil,108270_Prinny_Presents_NIS_Classics_Volume_2.jpg,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,38,1,80,0,0,Nintendo Switch,5,2022,Prinny Presents: NIS Classics - Volume 2,2022.0,Prinny Presents: NIS Classics Volume 2 - Delux...,prinny-presents-nis-classics-volume-2-deluxe-e...,NaN,NaN,Two action-packed RPG classics are making thei...,https://www.igdb.com/games/prinny-presents-nis...,None,single-player,"role-playing-rpg, strategy",None,switch,third-person,"action, fantasy",prinny-presents-nis-classics,NaN


### Pre-selecting what columns will be used for the models
Here i'm selecting solely based on what makes sense to be included or not. Later i'll use more robust techniques

In [31]:
columns = ['hltb_game_name', 'hltb_comp_main', 'hltb_comp_plus', 'hltb_comp_100', 'hltb_count_backlog', 'hltb_review_score', 'hltb_profile_popular', 'hltb_release_world', 'igdb_rating', 'igdb_game_modes', 'igdb_genres', 'igdb_keywords', 'igdb_platforms', 'igdb_player_perspectives', 'igdb_themes']

In [32]:
df_prd = df[columns]

### Applying Frequency Encoding to deal with categorical data

In [ ]:
def frequency_encoding(df_, col):
    df_[col] = df_[col].str.split(', ')

    all_categories = [cat for sublist in df_[col] for cat in sublist]
    freq_map = pd.Series(all_categories).value_counts(normalize=True)

    df_[f'{col}_fe'] = df_[col].apply(lambda x: sum(freq_map[cat] for cat in x) / len(x))

    return df_

In [36]:
df_prd.sample()

,hltb_game_name,hltb_comp_main,hltb_comp_plus,hltb_comp_100,hltb_count_backlog,hltb_review_score,hltb_profile_popular,hltb_release_world,igdb_rating,igdb_game_modes,igdb_genres,igdb_keywords,igdb_platforms,igdb_player_perspectives,igdb_themes
1236,Magna Braban: Henreki no Yusha,64920,0,0,17,60,9,1994,NaN,single-player,role-playing-rpg,"japan-only, fan-translation-english",sfam,bird-view-slash-isometric,fantasy


In [35]:
cat_cols = ['igdb_game_modes', 'igdb_genres', 'igdb_keywords', 'igdb_platforms', 'igdb_player_perspectives', 'igdb_themes']

In [ ]:
for cat_col in cat_cols:
    df_prd = frequency_encoding(df_=df_prd, col=cat_col)